In [9]:
import e3nn_jax as e3nn
from e3nn_jax._src.s2grid import s2_grid, _quadrature_weights_soft, to_s2grid
import jax
from jax import numpy as jnp
from matplotlib import pyplot as plt
import numpy as np
import typing
import plotly
import plotly.graph_objects as go

from sampling import sample_s2grid_angular

## sample from s2grid

In [2]:
res_alpha = 51
res_beta = 30
quadrature = "soft"
key = jax.random.PRNGKey(0)

In [4]:
z, alpha = s2_grid(res_beta, res_alpha, quadrature=quadrature)
z, alpha = np.meshgrid(z, alpha, indexing='ij')
x = jnp.cos(alpha) * jnp.sqrt(1 - z**2)
y = jnp.sin(alpha) * jnp.sqrt(1 - z**2)

c = e3nn.IrrepsArray("0e+1o+2e", jnp.array([0, 0, 0, 0, 0, 100.0, 0, 0, 0]))
f = e3nn.to_s2grid(c, (res_beta, res_alpha), quadrature=quadrature)


In [5]:
Z = e3nn.from_s2grid(jnp.exp(f), 1, p_val=1, p_arg=1, quadrature=quadrature).array[0]
p = jnp.exp(f) / Z

In [11]:
keys = jax.random.split(key, num=100)
sampled_z = []
sampled_alpha = []
for k in keys:
    z_k, a_k = sample_s2grid_angular(p, res_beta, res_alpha, quadrature=quadrature, key=k)
    sampled_z.append(z_k)
    sampled_alpha.append(a_k)
sampled_z = jnp.array(sampled_z)
sampled_alpha = jnp.array(sampled_alpha)
sampled_x = jnp.cos(sampled_alpha) * jnp.sqrt(1 - sampled_z**2)
sampled_y = jnp.sin(sampled_alpha) * jnp.sqrt(1 - sampled_z**2)

In [12]:
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, surfacecolor=f)])
fig.show()

In [13]:
fig = go.Figure(data=[go.Surface(x=x*0.8, y=y*0.8, z=z*0.8, surfacecolor=p), go.Scatter3d(x=sampled_x, y=sampled_y, z=sampled_z)])
fig.show()